In [27]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import functions.token_functions as tf
from langdetect import detect


import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim_models

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

from spacy.lang.en.stop_words import STOP_WORDS as stopwords

from collections import Counter, defaultdict

nlp = spacy.load('en_core_web_sm')

In [22]:
# Create an empty list to store the review dataframes
df_list = []
csv_files = os.listdir('../data/games/games',)

# Iterate over each file in the folder
for file in csv_files:
    # Read the CSV file into a datadrame
    df = pd.read_csv('../data/games/games/'+file, index_col=0)

    # Append the dataframe to the list
    df_list.append(df)

# Combine the dataframes into a single dataframe
games_df = pd.concat(df_list, ignore_index=True)


In [40]:
# drop values
games_df = games_df.dropna()

In [42]:
games_df['review_title'].apply(detect)

LangDetectException: No features in text.

In [29]:
games_df['review_tokens'] = games_df['review'].apply(tf.clean_tokenize)